In [ ]:
# kode som kunn finner utliggere

import json
import numpy as np

def find_outliers_in_weather_data(file_path, data_key='max_temp'):
    
    try:
        # Åpne og les JSON-filen
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Ekstraher verdiene for den spesifikke nøkkelen (for eksempel max_temp), hopp over None verdier
        sample_data_with_outliers = np.array([entry[data_key] for entry in data if entry[data_key] is not None])

        # Sett grenseverdier direkte
        lower_limit = -50
        upper_limit = 50

        # Finn uteliggere ved å bruke grenseverdiene
        outliers = sample_data_with_outliers[(sample_data_with_outliers < lower_limit) | (sample_data_with_outliers > upper_limit)]

        if len(outliers) > 0:
            return outliers
        else:
            print(f"Det er ingen uteliggere for '{data_key}' basert på de angitte grensene.")
            return None
    except Exception as e:
        print(f"Feil ved behandling av filen: {e}")
        return None

# Eksempel på hvordan du kan bruke funksjonen:
file_path = r'C:\anvendt_prog\Anvendt_prosjekt\data\london_weather.json'
outliers = find_outliers_in_weather_data(file_path, data_key='max_temp')

# Skriv ut uteliggerne
if outliers is not None:
    print("Outliers:", outliers)


Outliers: [200.3]


In [46]:
# Eksempel på hvordan du kan bruke funksjonen for min_temp
file_path = r'C:\anvendt_prog\Anvendt_prosjekt\data\london_weather.json'
outliers = find_outliers_in_weather_data(file_path, data_key='min_temp')

# Skriv ut uteliggerne
if outliers is not None:
    print("Outliers for min_temp:", outliers)


Outliers for min_temp: [-700.5]


In [ ]:
# kode som endrer på alle utligere og NONE verdier
# oppretter ny json fil ved kjøring
#Behalnding av data i london_weather

import pandas as pd
import pandasql as psql
import numpy as np
import json

# Funksjon for å finne uteliggere og håndtere manglende verdier for alle kolonner
def find_outliers_and_handle_missing(file_path, data_keys):
    try:
        # Les JSON-filen
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Konverter JSON til en Pandas DataFrame
        df = pd.DataFrame(data)
        
        # Iterer gjennom alle kolonnene som skal behandles
        for data_key in data_keys:
            # Bruk SQL-spørring for å finne NULL-verdier i den spesifikke kolonnen
            query = f"SELECT * FROM df WHERE {data_key} IS NULL"
            null_values = psql.sqldf(query, locals())
            
            if not null_values.empty:
                print(f"Found NULL values in '{data_key}':")
                print(null_values)
            else:
                print(f"No NULL values found in '{data_key}'.")

            # Beregn gjennomsnitt og standardavvik for uteliggere
            mean = np.mean(df[data_key].dropna())  # dropna() for å unngå None/NaN
            std_dev = np.std(df[data_key].dropna())
            
            # Definer grenseverdier for uteliggere (kan justeres for hver kolonne om ønskelig)
            lower_limit = df[data_key].quantile(0.05)  # 5%-percentilen som nedre grense
            upper_limit = df[data_key].quantile(0.95)  # 95%-percentilen som øvre grense
            
            # Finn uteliggere ved å bruke pandas
            outliers = df[(df[data_key] < lower_limit) | (df[data_key] > upper_limit)]
            
            if not outliers.empty:
                print(f"Outliers found in '{data_key}':")
                print(outliers)
            else:
                print(f"No outliers found in '{data_key}'.")
            
            # Erstatte NULL-verdier med gjennomsnittet og runde til to desimaler
            mean_rounded = round(mean, 2)  # Runder gjennomsnittet til 2 desimaler
            df.loc[df[data_key].isnull(), data_key] = mean_rounded
            print(f"Replaced NULL values in '{data_key}' with the mean value: {mean_rounded}")
            
            # Erstatte uteliggere med gjennomsnittet og runde til to desimaler
            df.loc[(df[data_key] < lower_limit) | (df[data_key] > upper_limit), data_key] = mean_rounded
            print(f"Replaced outliers in '{data_key}' with the mean value: {mean_rounded}")

            # Runder hele kolonnen til 2 desimaler
            df[data_key] = df[data_key].round(2)

        return df

    except Exception as e:
        print(f"Error processing the file: {e}")
        return None

# Funksjon for å lagre DataFrame til JSON-fil
def save_to_json(df, output_file):
    df_json = df.to_dict(orient='records')
    with open(output_file, 'w') as json_file:
        json.dump(df_json, json_file, indent=4)
    print(f"Updated data saved to {output_file}")

# Eksempel på hvordan du kan bruke funksjonene:
file_path = r'C:\anvendt_prog\Anvendt_prosjekt\data\london_weather.json'
output_file = r'C:\anvendt_prog\Anvendt_prosjekt\data\updated_london_weather.json'

# Kall funksjonen for å håndtere uteliggere og NULL-verdier for alle relevante kolonner
data_keys = ['cloud_cover', 'sunshine', 'global_radiation', 'max_temp', 'mean_temp', 'min_temp', 'precipitation', 'pressure', 'snow_depth']
df_cleaned = find_outliers_and_handle_missing(file_path, data_keys)

# Lagre den oppdaterte DataFrame-en til en ny JSON-fil
save_to_json(df_cleaned, output_file)


In [42]:
import pandas as pd

# Funksjon for å hente tmin, tmax og tavg fra CSV-fil
def get_min_max_temperature(file_path):
    try:
        # Les CSV-filen til en Pandas DataFrame og hopp over problematiske linjer
        df = pd.read_csv(file_path, on_bad_lines='skip')
        
        # Hent ut time, tmin, tmax og tavg kolonnene
        tmin_tmax_tavg = df[['time', 'tmin', 'tmax', 'tavg']]
        
        # Returner DataFrame med 'time', 'tmin', 'tmax', og 'tavg'
        return tmin_tmax_tavg

    except Exception as e:
        print(f"Error loading file: {e}")
        return None

# Teste funksjonen med en fil
file_path = r'C:\anvendt_prog\Anvendt_prosjekt\data\bostonData2.csv'  # Bruk rå streng (raw string) for filstien
min_max_data = get_min_max_temperature(file_path)

if min_max_data is not None:
    print(min_max_data)

# Skriv ut resultatet
print(get_min_max_temperature(file_path))


            time  tmin  tmax  tavg
0     2013-03-01   1.1   5.0   3.2
1     2013-03-02   1.1   5.6   3.1
2     2013-03-03   0.6   6.1   2.6
3     2013-03-04  -0.6   5.0   1.8
4     2013-03-05   0.0   6.1   2.9
...          ...   ...   ...   ...
3648  2023-02-25 -10.5  -6.6  -7.9
3649  2023-02-26  -7.7   3.9  -4.2
3650  2023-02-27  -2.7  -0.5  -1.7
3651  2023-02-28  -1.6   2.2   0.5
3652  2023-03-01   0.6   5.6   2.4

[3653 rows x 4 columns]
            time  tmin  tmax  tavg
0     2013-03-01   1.1   5.0   3.2
1     2013-03-02   1.1   5.6   3.1
2     2013-03-03   0.6   6.1   2.6
3     2013-03-04  -0.6   5.0   1.8
4     2013-03-05   0.0   6.1   2.9
...          ...   ...   ...   ...
3648  2023-02-25 -10.5  -6.6  -7.9
3649  2023-02-26  -7.7   3.9  -4.2
3650  2023-02-27  -2.7  -0.5  -1.7
3651  2023-02-28  -1.6   2.2   0.5
3652  2023-03-01   0.6   5.6   2.4

[3653 rows x 4 columns]


In [44]:
import json

# Funksjon for å hente max, mean og min temperatur fra JSON-fil
def get_temperatures_from_json(file_path):
    try:
        # Åpne og last inn JSON-data fra filen
        with open(file_path, 'r') as file:
            data = json.load(file)
        
        # Sjekk om data er en liste (som kan inneholde flere objekter)
        if isinstance(data, list):
            temperatures = []
            for entry in data:
                # Hent ut max_temp, mean_temp og min_temp fra hvert objekt i listen
                temp_data = {
                    "max_temp": entry.get("max_temp"),
                    "mean_temp": entry.get("mean_temp"),
                    "min_temp": entry.get("min_temp")
                }
                temperatures.append(temp_data)
        else:
            # Hvis data ikke er en liste, hent bare én verdi (som tidligere)
            temperatures = {
                "max_temp": data.get("max_temp"),
                "mean_temp": data.get("mean_temp"),
                "min_temp": data.get("min_temp")
            }

        # Returner temperaturene
        return temperatures
    
    except Exception as e:
        print(f"Error loading file: {e}")
        return None

# Eksempel på hvordan du kan bruke funksjonen
file_path = r'C:\anvendt_prog\Anvendt_prosjekt\data\updated_london_weather.json'  # Sett inn riktig filsti
temperatures = get_temperatures_from_json(file_path)

if temperatures:
    print(temperatures)


[{'max_temp': 15.4, 'mean_temp': 11.48, 'min_temp': 7.51}, {'max_temp': 15.4, 'mean_temp': 11.48, 'min_temp': 7.51}, {'max_temp': 15.4, 'mean_temp': 11.48, 'min_temp': 7.51}, {'max_temp': 15.4, 'mean_temp': 11.48, 'min_temp': 7.51}, {'max_temp': 5.6, 'mean_temp': 11.48, 'min_temp': 7.51}, {'max_temp': 8.3, 'mean_temp': 11.48, 'min_temp': 7.51}, {'max_temp': 8.5, 'mean_temp': 11.48, 'min_temp': 7.51}, {'max_temp': 5.8, 'mean_temp': 6.9, 'min_temp': 5.3}, {'max_temp': 5.2, 'mean_temp': 3.7, 'min_temp': 1.6}, {'max_temp': 15.4, 'mean_temp': 3.3, 'min_temp': 1.4}, {'max_temp': 15.4, 'mean_temp': 2.6, 'min_temp': 0.3}, {'max_temp': 15.4, 'mean_temp': 11.48, 'min_temp': 7.51}, {'max_temp': 15.4, 'mean_temp': 11.48, 'min_temp': 7.51}, {'max_temp': 6.7, 'mean_temp': 11.48, 'min_temp': 7.51}, {'max_temp': 5.9, 'mean_temp': 3.8, 'min_temp': 1.0}, {'max_temp': 15.4, 'mean_temp': 5.0, 'min_temp': 4.1}, {'max_temp': 15.4, 'mean_temp': 11.48, 'min_temp': 1.6}, {'max_temp': 15.4, 'mean_temp': 11.48, 